In [ ]:
# base_packs
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn.preprocessing as skp
import sklearn.decomposition as skd
import statsmodels.api as sm
import sklearn.manifold as skm
import sklearn.cluster as skc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from statsmodels.stats.diagnostic import het_white
import ruptures as rpt
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [ ]:
dt = "data"
dt = pd.read_excel(dt)
dt.columns.to_numpy

In [ ]:
dt.shape

In [ ]:
dt.dtypes

In [ ]:
dt[''].replace(',' , '.', inplace = True)
dt['']= pd.to_numeric(dt[''] , errors = 'coerce')
dt[''].count()

In [ ]:
dt.drop(columns=[''], inplace=True)
dt.replace('', 1 , inplace = True)
dt.replace('', 0 , inplace = True)

In [ ]:
dt = pd.get_dummies(dt)

In [ ]:
dt[''] = dt[''] + dt[''] + dt['']

In [ ]:
dt

In [ ]:
#substituir nan  pela mediana.
for i in dt.columns:
    median_value = dt[i].median()  # Calculate median for the column
    dt[i].fillna(median_value, inplace=True)

In [ ]:
dt = dt.replace({False: 0, True: 1})
dt

In [ ]:
contr = dt['Pressão ...']
contr.values

In [ ]:
contr_dt = pd.DataFrame(contr.values)
contr_dt

In [ ]:
model = rpt.Pelt(model="rbf").fit(contr.values)
breakpoints = model.predict(pen= 0)
breaks_df = pd.DataFrame({
    'Break_Index': breakpoints,
     
    
})

In [ ]:
clusters = []
for i in range(len(breakpoints) - 1):
    clusters.append((breakpoints[i], breakpoints[i+1]))


In [ ]:
dt['Tag_processo_pressão'] = np.nan
for i, (start, end) in enumerate(clusters):
    dt.loc[dt.index[start:end], 'Tag_processo_pressão'] = i + 1 

In [ ]:
dt['Tag_processo_pressão'] = dt['Tag_processo_pressão'].fillna(0)
dt['Tag_processo_pressão']

In [ ]:
clus_tab = dt.groupby(['Tag_processo_pressão']).mean()
clus_tab['Tempo ...'] 

In [ ]:
clus_tab

In [ ]:
## Clusterização e regressão logística para ver as causas de uma liga boa e ruim.
#separação em Teor de C após a descarburação(%), menor que a media do dt, e Tempo total de descarburação(min)
##menor que a média do dt.
# regressão dá a significância e impacto, clus_tab dá  a combinação de variáveis.
#criar outra variavel clusters com a reclassificação para a regressão logistica, 0 e 1, assim fica mais fácil de pegar as combinaçõs do processo.

In [ ]:
cluster= pd.DataFrame(dt['Tag_processo_pressão'])
cluster

In [ ]:
dt['cluster'] = 0
dt.loc[(dt['Teor de C após a descarburação(%)'] < dt['Teor de C após a descarburação(%)'].mean() ) & (dt['tempo_total'] < dt['tempo_total'].mean()), 'cluster'] = 1

In [ ]:
dt['cluster'].value_counts()

In [ ]:
tag = pd.DataFrame(dt['Tag_processo_pressão'])
tag = tag.astype(int).astype(str)
tag

In [ ]:
tag = pd.get_dummies(tag)
tag.replace({False: 0, True: 1})
tag

In [ ]:
scaler = StandardScaler()
dt_scaled = scaler.fit_transform(dt)
dt_scaled = pd.DataFrame(dt_scaled, columns=dt.columns )
dt_scaled.columns

In [ ]:
## rodar duas regressões, uma só com as tag_processo pressão em dummie, e outras sem as tags.

In [ ]:
#Logit:
x_ = dt_scaled[[
      'all the features expect the target']]
x_x = tag[tag.columns]


y_ = dt['cluster']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_x, y_, test_size=0.2, random_state=42)

In [ ]:
# Decision tree classifier:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(x_x, y_)


In [ ]:
from sklearn.inspection import permutation_importance
result = permutation_importance(clf, x_x, y_, n_repeats=10, random_state=42)
importance = result.importances_mean
importance = pd.DataFrame(importance)
importance

In [ ]:
from sklearn.linear_model import LogisticRegression
import statsmodels.discrete.discrete_model as sm
model= sm.Logit(y_,x_)
co =model.fit().params
coe = pd.DataFrame(co)
coe.columns = ['coeff']
exp_coeff =   np.exp(coe)/(np.exp(coe) + 1) 
exp_coeff = pd.DataFrame(exp_coeff)
exp_coeff.columns = ['exp_coeff']
exp_coeff
results = model.fit( )
results_summary = results.summary()
results_summary

In [ ]:
results_as_html = results_summary.tables[1].as_html()
df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
df_results['exp_coeff'] = exp_coeff
df_results

In [ ]:
## Temos qual processo, clf, e oque alterar nesse processo, regressão logística.

In [ ]:
# Em suma, agrupamos via quebra-estrutural da tag do processo de descarburação do forno VOD.
# Dessa forma, selecionamos os processos, clusters com menor teor de C e menor tempo de processo, com cluster igual a 1
# Rodamos uma regressão logística com os dois grupos, dessa forma temos os fatores que de fato causam as características que queremos otimizar.
# e rodamos uma classifier tree com as dummies dos processos,tag_processos, para selecionar qual o processo que favorece as caracter´siticas que buscamos.
# As características sendo o baixo teor de C na liga e menor tempode processo, em vista a otimizar o forno.